In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
dataset_path = "/content/gdrive/MyDrive/Memory Retention Research/Datasets/Dataset_10000/Dataset_10000.csv"
train_path = "/content/gdrive/MyDrive/Memory Retention Research/Datasets/Dataset_5000/train.csv"
test_path = "/content/gdrive/MyDrive/Memory Retention Research/Datasets/Dataset_5000/test.csv"
val_path = "/content/gdrive/MyDrive/Memory Retention Research/Datasets/Dataset_5000/validation.csv"
base_path = "/content/gdrive/MyDrive/Memory Retention Research/Datasets/"

In [ ]:
!pip install datasets
!pip install fairseq
!pip install transformers
!pip install contractions
!pip install sentencepiece
!pip install transliterate

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import torch
import keras
import nltk
import nltk.data
import contractions
import re
import random
import copy
import string
import time, datetime
from tqdm import tqdm
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datasets as ds
import gensim.downloader as api

from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.models import Sequential, Model, clone_model
from keras.layers import Input, Concatenate, Dense, Conv1D, GlobalMaxPooling1D, Reshape, Embedding, LSTM, Dropout, Multiply, Dot, Activation
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping
from keras.initializers import Constant
from keras.optimizers import Adam, Adagrad, Adadelta

from transformers import RobertaTokenizer, RobertaModel
from transformers import BertModel, BertTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import T5Tokenizer, T5Model

from gensim.models.doc2vec import Doc2Vec, TaggedDocument, Word2Vec
from gensim.models import KeyedVectors

from nltk import word_tokenize
from nltk.corpus import stopwords, words

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
#for glove
glove_model = api.load("glove-wiki-gigaword-300")

In [ ]:
class DatasetCreation():
  def __init__(self, glove_model, min_word_size=10000):

    self.vocabulary = set()
    self.positive_vocabulary = list()
    self.stop_words = set(stopwords.words('english'))
    self.english_words = set(words.words())
    self.min_word_size = min_word_size
    self.glove_model = glove_model

    with open('bias_lexicon.txt', 'r') as file:
      self.bias_words = file.read().splitlines()

    self.vad_df = pd.read_csv('NRC-VAD-Lexicon.txt', delimiter='\t', names=['Words', 'valence', 'arousal', 'dominance'])
    self.vad_words = set(self.vad_df['Words'].values)
    self.bias_words = set(self.bias_words)

    self.vad_bias_words = list(self.vad_words.union(self.bias_words))

    self.final_word_mapping = {}

    self.vad_bias_dict = {}
    for vad_bias in self.vad_bias_words:
      self.vad_bias_dict[vad_bias] = []

  def load_data(self):
    self.dataset = ds.load_dataset('lambada')
    self.trainset = self.dataset['train']['text']
    self.testset = self.dataset['test']['text']
    self.dataset = self.trainset + self.testset
    print("{} paragraphs.\n".format(len(self.dataset)))

  def create_paragraphs(self):
    times = 2
    time = 0
    result_cleaned_paragraphs = []
    cleaned_paragraphs = self.df['cleaned_paragraph'].values
    for paragraph in cleaned_paragraphs:
      time = 0
      res_paragraph = []
      sentences = nltk.sent_tokenize(paragraph)
      for sentence in sentences:
        tokens = sentence.split()
        if(len(res_paragraph) <= self.min_word_size):
          res_paragraph.extend(tokens)
        else:
          result_cleaned_paragraphs.append(' '.join(res_paragraph))
          if time < times:
            time += 1
            res_paragraph = []
            res_paragraph.extend(tokens)
            continue
          else:
            break
    return result_cleaned_paragraphs


  def create_dataframe(self):
    self.df = pd.DataFrame(self.dataset, columns=['raw_paragraph'])

  def is_meaningful(self, word):
    return word in self.english_words and word not in self.stop_words

  def clean_paragraph(self, paragraph):
    text = re.sub('\s+', ' ', paragraph.strip())
    text = text.lower()
    return text

  def create_vocabulary(self, paragraph):
    words = nltk.word_tokenize(paragraph)
    self.vocabulary.update(words)

  def has_negative_word(self, paragraph, positive_word):
    if positive_word in self.glove_model.key_to_index:
      similar_words = self.glove_model.most_similar(positive_word)
      for similar_word, _ in similar_words:
        if similar_word not in paragraph:
          return True
    return False

  def complete_vad_bias_dictionary(self, cleaned_paragraphs):
    all_len = len(self.vad_bias_dict.keys())
    for i, paragraphs in enumerate(cleaned_paragraphs):
      tokens = paragraphs.split()
      intersect = set(tokens).intersection(self.vad_bias_words)
      for vad_bias_word in list(intersect):
        self.vad_bias_dict[vad_bias_word].append(i)
    empty_keys = [key for key, value in self.vad_bias_dict.items() if len(value) == 0]
    for key in empty_keys:
      del self.vad_bias_dict[key]
    modified_len = len(self.vad_bias_dict.keys())
    print("{} words are in paragraphs out of {}".format(modified_len, all_len))

  def delete_index_from_all_dictionary(self, index):
    delete_these = []
    for key, value in self.vad_bias_dict.items():
      if index in value:
        delete_these.append(key)
    for key in delete_these:
      self.vad_bias_dict[key].remove(index)

    empty_keys = [key for key, value in self.vad_bias_dict.items() if len(value) == 0]
    for key in empty_keys:
      del self.vad_bias_dict[key]


  def assign_vad_bias(self):
    sorted_dict = dict(sorted(self.vad_bias_dict.items(), key=lambda item: len(item[1])))
    while(len(sorted_dict) != 0):
      key = list(sorted_dict.keys())[0]
      indexes = sorted_dict[key]
      index = random.choice(indexes)
      del self.vad_bias_dict[key]
      self.final_word_mapping[index] = key
      self.delete_index_from_all_dictionary(index)
      sorted_dict = dict(sorted(self.vad_bias_dict.items(), key=lambda item: len(item[1])))

  def random_positive_word(self, paragraph):
    searching = True
    dictionary = np.unique(paragraph.split())
    while(searching):
      positive_word = random.choice(dictionary)
      if self.is_meaningful(positive_word) and self.has_negative_word(paragraph, positive_word):
        return positive_word


  def fill_positive_words(self, paragraphs):
    self.positive_words = [0 for _ in range(len(paragraphs))]
    for i, paragraph in enumerate(paragraphs):
      if i in self.final_word_mapping.keys():
        self.positive_words[i] = self.final_word_mapping[i]
      else:
        self.positive_words[i] = self.random_positive_word(paragraph)

  def find_positive_words(self, cleaned_paragraphs):
    self.complete_vad_bias_dictionary(cleaned_paragraphs)
    self.assign_vad_bias()
    self.fill_positive_words(cleaned_paragraphs)

  def find_negative_words(self, paragraphs):
    self.negative_words = [0 for _ in range(len(self.positive_words))]
    for i, paragraph in tqdm(enumerate(paragraphs)):
      positive_word = self.positive_words[i]
      if positive_word in self.glove_model.key_to_index:
        similar_words = self.glove_model.most_similar(positive_word)
      else:
        continue
      for neg_word, _ in similar_words:
        if neg_word not in paragraph:
          self.negative_words[i] = neg_word
          break
    for i, negative_word in tqdm(enumerate(self.negative_words)):
      if negative_word == 0:
        positive_word = self.random_positive_word(paragraph)
        similar_words = self.glove_model.most_similar(positive_word)
        for neg_word, _ in similar_words:
          if neg_word not in paragraph:
            self.negative_words[i] = neg_word
            break
  def drop_duplicates(self):
    searching = True
    dup = self.df.drop_duplicates(subset=['Target Word'], keep='first')
    while (searching):
      pos_df = dup[dup['Label']==1]
      neg_df = dup[dup['Label']==0]
      print("we have {} positive unique words".format(len(pos_df)))
      print("we have {} negative unique words".format(len(neg_df)))
      if len(pos_df) == len(neg_df):
        break

      if len(neg_df) < len(pos_df):
        pos_df = pos_df[pos_df['Paragraph'].isin(neg_df['Paragraph'])]
      else:
        neg_df = neg_df[neg_df['Paragraph'].isin(pos_df['Paragraph'])]
      dup = pd.concat([pos_df, neg_df])
    self.df = dup

  def save(self):
    base_path = "/content/gdrive/MyDrive/Memory Retention Research/Datasets/Dataset_{}/Dataset_{}.csv".format(self.min_word_size, self.min_word_size)
    self.df.to_csv(base_path, index=False)

  def create(self):
    print("Creating..")
    self.df['cleaned_paragraph'] = self.df['raw_paragraph'].apply(lambda paragraph:self.clean_paragraph(paragraph))
    print("Paragraph Cleaned!")

    cleaned_paragraphs = self.create_paragraphs()
    print("We have {} paragraphs.".format(len(cleaned_paragraphs)))
    cleaned_paragraphs = np.unique(cleaned_paragraphs)
    print("Now we have {} unique paragraphs.".format(len(cleaned_paragraphs)))

    self.find_positive_words(cleaned_paragraphs)
    print("Positive words found. We have {} positive words.".format(len(self.positive_words)))
    self.find_negative_words(cleaned_paragraphs)
    print("Negative words found. We have {} negative words.".format(len(self.negative_words)))


    length = len(cleaned_paragraphs)
    label_1 = [1 for _ in range(length)]
    label_0 = [0 for _ in range(length)]

    cleaned_paragraphs = list(cleaned_paragraphs)
    self.positive_words = list(self.positive_words)
    self.negative_words = list(self.negative_words)

    cleaned_paragraphs.extend(cleaned_paragraphs)
    self.positive_words.extend(self.negative_words)
    label_1.extend(label_0)

    data = {
    'Paragraph': cleaned_paragraphs,
    'Target Word': self.positive_words,
    'Label': label_1
    }
    self.df = pd.DataFrame(data)
    self.drop_duplicates()

  def start(self):
    self.load_data()
    self.create_dataframe()
    self.create()
    print("Creation is DONE")
    self.save()
    print("Dataframe is SAVED")
    print()

In [ ]:
dataset = DatasetCreation(glove_model, 10000)

In [ ]:
dataset.start()

In [ ]:
dataset.df.describe()

,Label
count,11158.000000
mean,0.500000
std,0.500022
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['Label'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_1, *['Label'], **{})
chart

In [ ]:
class Statistics():
  def __init__(self, filename, glove_model):
    self.df = pd.read_csv(filename)
    self.path = "/content/gdrive/MyDrive/Memory Retention Research/Datasets/Dataset_10000/Dataset_10000_temp.csv"

    null_rows = self.df.isnull().any(axis=1)
    rows_with_null_data = self.df[null_rows]
    paragraph = rows_with_null_data['Paragraph']
    self.df = self.df[~self.df['Paragraph'].isin(paragraph)]
    self.glove_model = glove_model

    with open('bias_lexicon.txt', 'r') as file:
      self.bias_words = file.read().splitlines()

    self.vad_df = pd.read_csv('NRC-VAD-Lexicon.txt', delimiter='\t', names=['Words', 'valence', 'arousal', 'dominance'])
    self.vad_words = set(self.vad_df['Words'].values)

  def calculate_C(self):
    C = []
    paragraphs = self.df['Paragraph'].values
    for paragraph in paragraphs:
      tokens = paragraph.split()
      C.append(len(tokens))
    return C

  def calculate_d(self):
    d = []
    paragraphs = self.df['Paragraph'].values
    target_words = self.df['Target Word'].values
    for i, paragraph in enumerate(paragraphs):
      tokens = paragraph.split()
      length = len(tokens)
      if target_words[i] in tokens:
        ind = tokens.index(target_words[i])
        diff = length - ind - 1
      else:
        diff = -1
      d.append(diff)
    return d

  def calculate_most_similar_and_similarity(self):
    sim_words = []
    sim_scores = []
    target_words = self.df['Target Word'].values
    for target_word in tqdm(target_words):
      if target_word in self.glove_model.key_to_index:
        sim_word, sim_score = self.glove_model.most_similar(target_word)[0]
        sim_words.append(sim_word)
        sim_scores.append(sim_score)
      else:
        sim_words.append(-1)
        sim_scores.append(-1)

    assert len(target_words) == len(sim_words)

    return sim_words, sim_scores

  def calculate_repetition(self):
    repetition = []
    paragraphs = self.df['Paragraph'].values
    target_words = self.df['Target Word'].values

    for i, paragraph in enumerate(paragraphs):
      repeat = paragraph.split().count(target_words[i])
      repetition.append(repeat)
    return repetition

  def is_vad(self):
    is_vad = []

    target_words = self.df['Target Word'].values
    for target_word in target_words:
      if target_word in self.vad_words:
        is_vad.append(1)
      else:
        is_vad.append(0)

    return is_vad

  def is_bias(self):
    is_bias = []

    target_words = self.df['Target Word'].values
    for target_word in target_words:
      if target_word in self.bias_words:
        is_bias.append(1)
      else:
        is_bias.append(0)

    return is_bias

  def calculate_POS(self):
    poses = []

    paragraphs = self.df[self.df['Label']==1]['Paragraph'].values
    target_words = self.df[self.df['Label']==1]['Target Word'].values

    for i, paragraph in tqdm(enumerate(paragraphs)):
      words = nltk.word_tokenize(paragraph)
      pos_tags = nltk.pos_tag(words)

      for word, pos in pos_tags:
        if word == target_words[i]:
          poses.append(pos)
          break

    neg_poses = [-1 for _ in range(len(poses))]
    poses.extend(neg_poses)
    return poses

  def calculate_unigram_frequency(self):
    word_freq = []

    paragraphs = np.unique(self.df['Paragraph'].values)
    paragraphs = list(paragraphs)
    document = ' '.join(paragraphs)

    words = document.split()
    word_frequency = Counter(words)

    target_words = self.df['Target Word'].values
    for word in target_words:
      if word in word_frequency:
        word_freq.append(word_frequency[word])
      else:
        word_freq.append(0)
    return word_freq

  def calculate_query_length(self):
    query_length = []
    words = self.df['Target Word'].values
    for query in words:
      query_length.append(len(query))
    return query_length

  def save(self, filename, base_path):
    self.df.to_csv(filename, index=False)

    pos_df = self.df[self.df['Label']==1]
    neg_df = self.df[self.df['Label']==0]

    pos_train_df, pos_remaining = train_test_split(pos_df, test_size=0.1, random_state=42)
    pos_valid_df, pos_test_df = train_test_split(pos_remaining, test_size=0.5, random_state=42)

    neg_train_df, neg_remaining = train_test_split(neg_df, test_size=0.1, random_state=42)
    neg_valid_df, neg_test_df = train_test_split(neg_remaining, test_size=0.5, random_state=42)

    train_df = pd.concat([pos_train_df, neg_train_df])
    test_df = pd.concat([pos_test_df, neg_test_df])
    val_df = pd.concat([pos_valid_df, neg_valid_df])

    train_df.to_csv( base_path + "Dataset_10000/train.csv", index=False)
    test_df.to_csv(base_path + "Dataset_10000/test.csv", index=False)
    val_df.to_csv(base_path + "Dataset_10000/val.csv", index=False)

  def save_periodically(self):
    self.df.to_csv(self.path, index=False)

  def run_statistics(self):
    self.df['C'] = self.calculate_C()
    self.save_periodically()
    print("C is calculated and saved")

    self.df['d'] = self.calculate_d()
    self.save_periodically()
    print("d is calculated and saved")

    self.df['most_similar'], self.df['similarity_score'] = self.calculate_most_similar_and_similarity()
    self.save_periodically()
    print("most similar and similarity is calculated and saved")

    self.df['POS'] = self.calculate_POS()
    self.save_periodically()
    print("POS is calculated and saved")

    self.df['repetitions'] = self.calculate_repetition()
    self.save_periodically()
    print("repetitions is calculated and saved")

    self.df['is_vad'] = self.is_vad()
    self.save_periodically()
    print("vad is calculated and saved")

    self.df['is_bias'] = self.is_bias()
    self.save_periodically()
    print("bias is calculated and saved")

    self.df['unigram_frequency'] = self.calculate_unigram_frequency()
    self.save_periodically()
    print("unigram_frequency is calculated and saved")

    self.df['query_length'] = self.calculate_query_length()
    self.save_periodically()
    print("length of query is calculated and saved")

In [ ]:
print(dataset_path)

/content/gdrive/MyDrive/Memory Retention Research/Datasets/Dataset_10000/Dataset_10000.csv


In [ ]:
statistics = Statistics(dataset_path, glove_model)

In [ ]:
statistics.run_statistics()

C is calculated and saved
d is calculated and saved


100%|██████████| 11158/11158 [12:38<00:00, 14.71it/s]


most similar and similarity is calculated and saved


5579it [55:20,  1.68it/s]


POS is calculated and saved
repetitions is calculated and saved
vad is calculated and saved
bias is calculated and saved
unigram_frequency is calculated and saved
length of query is calculated and saved


In [ ]:
statistics.save(dataset_path, base_path)

In [ ]:
class Embedding():
  def __init__(self, models, tokenizers):
    # self.glove_model, self.bert_model, self.roberta_model, self.gpt_model, self.T5_model = models
    self.glove_model = models
    # self.bert_tokenizer, self.roberta_tokenizer, self.gpt_tokenizer, self.T5_tokenizer = tokenizers

    self.VECTOR_SIZE = 300
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  def glove_representation(self, text):
    tokens = text.split()
    doc_vec = np.zeros((len(tokens), 300))
    for i, token in enumerate(tokens):
        if token in glove_model.key_to_index:
          doc_vec[i] = glove_model[token]
    doc_vec = np.mean(doc_vec, axis=0, keepdims=True)
    return doc_vec

  def bert_representation(self, text):
    chunk_size=500
    tokens = text.split()
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    doc_vec = []
    for chunk in chunks:
      input_ids = torch.tensor(self.bert_tokenizer.encode(chunk)).unsqueeze(0).to(self.device)
      outputs = self.bert_model(input_ids)
      last_hidden_states = outputs[0]
      cls_head = last_hidden_states[:, 0, :]
      doc_vec.append(cls_head)
    concatenated_doc_vec = torch.cat(doc_vec, dim=0)
    mean_doc_vec = torch.mean(concatenated_doc_vec, dim=0)
    return mean_doc_vec.detach().cpu().numpy().reshape(1,-1)

  def roberta_representation(self, text):
    chunk_size=500
    tokens = text.split()
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    doc_vec = []
    for chunk in chunks:
      input_ids = torch.tensor(self.roberta_tokenizer.encode(chunk)).unsqueeze(0).to(self.device)
      outputs = self.roberta_model(input_ids)
      last_hidden_states = outputs[0]
      cls_head = last_hidden_states[:, 0, :]
      doc_vec.append(cls_head)
    concatenated_doc_vec = torch.cat(doc_vec, dim=0)
    mean_doc_vec = torch.mean(concatenated_doc_vec, dim=0)
    return mean_doc_vec.detach().cpu().numpy().reshape(1,-1)

  def gpt_representation(self, text):
    tokens = self.gpt_tokenizer.tokenize(text)
    token_ids = self.gpt_tokenizer.convert_tokens_to_ids(tokens)
    pretrained_embeddings = self.gpt_model.transformer.wte.weight
    embeddings = pretrained_embeddings[token_ids]
    sentence_embedding = torch.mean(embeddings, dim=0)
    return sentence_embedding.cpu().detach().numpy()

  def T5_representation(self, text):
    chunk_size=500
    tokens = text.split()
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    doc_vec = []
    for chunk in chunks:
      input_ids = torch.tensor(self.T5_tokenizer.encode(chunk)).unsqueeze(0).to(self.device)
      outputs = self.T5_model.encoder(input_ids=input_ids)
      last_hidden_states = outputs.last_hidden_state
      cls_head = last_hidden_states[:, 0, :]
      doc_vec.append(cls_head)
    concatenated_doc_vec = torch.cat(doc_vec, dim=0)
    mean_doc_vec = torch.mean(concatenated_doc_vec, dim=0)
    return mean_doc_vec.detach().cpu().numpy().reshape(1, -1)

  def embed(self, df, technique='glove'):
    if technique=='glove':
      df['paragraph_representation'] = df['Paragraph'].apply(lambda text:self.glove_representation(text))
      print("Paragraph Representation Done.")

      df['word_representation'] = df['Target Word'].apply(lambda word:self.glove_representation(word))
      print("Word Representation Done.")

    elif technique=='bert':
      df['paragraph_representation'] = df['Paragraph'].apply(lambda text:self.bert_representation(text))
      print("Paragraph Representation Done.")

      df['word_representation'] = df['Target Word'].apply(lambda word:self.bert_representation(word))
      print("Word Representation Done.")

    elif technique=='roberta':
      self.df['paragraph_representation'] = self.df['Paragraph'].apply(lambda text:self.roberta_representation(text))
      print("Paragraph Representation Done.")

      self.df['word_representation'] = self.df['Target Word'].apply(lambda word:self.roberta_representation(word))
      print("Word Representation Done.")

    elif technique=='gpt':
      self.df['paragraph_representation'] = self.df['Paragraph'].apply(lambda text:self.gpt_representation(text))
      print("Paragraph Representation Done.")

      self.df['word_representation'] = self.df['Target Word'].apply(lambda word:self.gpt_representation(word))
      print("Word Representation Done.")

    elif technique=='T5':
      self.df['paragraph_representation'] = self.df['Paragraph'].apply(lambda text:self.T5_representation(text))
      print("Paragraph Representation Done.")

      self.df['word_representation'] = self.df['Target Word'].apply(lambda word:self.T5_representation(word))
      print("Word Representation Done.")

    elif technique=='lstm':
      with open('paragraphs.txt', 'w') as file:
        for paragraph in self.df['Paragraph']:
          file.write(paragraph + '\n')

      !bash LASER/tasks/embed/embed.sh ./paragraphs.txt ./paragraph_rep.bin

      dim = 1024
      X = np.fromfile("./paragraph_rep.bin", dtype=np.float32, count=-1)
      X.resize(X.shape[0] // dim, dim)
      self.df['paragraph_representation'] = [X[i].reshape(1, -1) for i in range(X.shape[0])]

      self.merge_positive_and_negative()
      print("Merge Done.")

      with open('target_words.txt', 'w') as file:
        for paragraph in self.df['target_word']:
          file.write(paragraph + '\n')

      !bash LASER/tasks/embed/embed.sh ./target_words.txt ./word_rep.bin

      X = np.fromfile("./word_rep.bin", dtype=np.float32, count=-1)
      X.resize(X.shape[0] // dim, dim)
      self.df['word_representation'] = [X[i].reshape(1, -1) for i in range(X.shape[0])]

    return df

  def run(self, dfs, technique='glove'):
    self.train = self.embed(dfs['train'], technique)
    self.test = self.embed(dfs['test'], technique)
    self.val = self.embed(dfs['val'], technique)
    print("Everything DONE!")
    print()

  def get_inputs(self, df, embed_dim):

    return self.train, self.test, self.val

    # X1 = np.array(df['paragraph_representation'].values.tolist()).reshape(-1, embed_dim)
    # X2 = np.array(df['word_representation'].values.tolist()).reshape(-1, embed_dim)
    # y = np.array(df['Label'].values.tolist()).reshape(-1, 1)

    # return (X1, X2, y)

In [ ]:
data = Embedding(glove_model, 0)

In [ ]:
data.run(dfs, technique='glove')

Paragraph Representation Done.
Word Representation Done.
Paragraph Representation Done.
Word Representation Done.
Paragraph Representation Done.
Word Representation Done.
Everything DONE!



In [ ]:
# train = data.get_inputs(data.train, 300)
# test = data.get_inputs(data.test, 300)
# val = data.get_inputs(data.val, 300)

train, test, val = data.get_inputs(data.train, 0)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch import nn
import torch.optim as optim

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, df, transform=None):
    self.df = df
    self.transform = transform

    self.paragraph_rep = self.df['paragraph_representation']
    self.word_rep = self.df['word_representation']
    self.label = self.df['Label']

  def __len__(self):
    return self.df

  def __getitem__(self, index):
    paragraph_rep = torch.tensor(self.paragraph_rep[index])
    word_rep = torch.tensor(self.word_rep[index])
    label = torch.tensor(self.label[index])


    return paragraph_rep, word_rep, label

In [ ]:

# Assuming you have a pandas DataFrame containing your data, e.g., 'data_df'
input1_data = torch.tensor(train['paragraph_representation'])
input2_data = torch.tensor(train['word_representation'])
label_data = torch.tensor(train['Label'])

input1_data = torch.reshape(input1_data, (-1, 300))
input2_data = torch.reshape(input2_data, (-1, 300))
label_data = torch.reshape(label_data, (-1, 1))

# Create a TensorDataset from your input data
dataset = TensorDataset(input1_data, input2_data, label_data)

# Create a DataLoader to iterate over the data in batches
batch_size = 64  # Adjust the batch size as per your requirements
training_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Now you can use the for loop as you initially intended
for batch_idx, (input1, input2, label) in enumerate(training_loader):
    # Your training code here
    pass


In [ ]:
trainDataset = CustomDataset(train)
testDataset = CustomDataset(test)
valDataset = CustomDataset(val)

In [ ]:
training_loader = DataLoader(trainDataset, batch_size=25, shuffle=False)
testing_loader = DataLoader(testDataset, batch_size=25, shuffle=False)
validation_loader = DataLoader(valDataset, batch_size=25, shuffle=False)

In [ ]:
epochs = 20
learning_rate = 0.1

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
class MemoryModel(nn.Module):
    def __init__(self, input1_dim, input2_dim, hidden_dim, output_dim, dropout_rate):
        super(MemoryModel, self).__init__()
        total_input_dim = input1_dim + input2_dim

        self.dense = nn.Linear(total_input_dim, hidden_dim)
        self.relu = nn.ReLU()

        self.dropout = nn.Dropout(p=dropout_rate)

        self.output_layer = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input1, input2):
        input1 = input1.to(torch.float32)
        input2 = input2.to(torch.float32)

        inputs = torch.cat((input1, input2), dim=1)
        inputs_dot = torch.tensor([torch.dot(torch.flatten(input1), torch.flatten(input2))])

        out = self.dense(inputs)
        out = self.relu(out)
        out = self.dropout(out)
        out = torch.cat((torch.flatten(out), inputs_dot), dim=0)

        output = self.output_layer(out)
        output = self.sigmoid(output)

        return output


In [ ]:
model = MemoryModel(300, 300, 300, 1, 0.3)
print(model)

for epoch in range(epochs):
  for batch_idx, (input1, input2, label) in enumerate(training_loader):

    scores = model(input1, input2)
    loss = criterion(scores, torch.flatten(label))

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

MemoryModel(
  (dense): Linear(in_features=600, out_features=300, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (output_layer): Linear(in_features=300, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


IndexError: ignored

In [ ]:
hyperparameters_list = []
optimizers = ['Adam']
activations = ['relu']
units = [300]
dropout_rates = [0.1, 0.3, 0.5, 0.8]
for optimizer in optimizers:
      for activation in activations:
        for unit in units:
          for dropout_rate in dropout_rates:
            hyperparameters_list.append((optimizer, activation, unit, dropout_rate))

In [ ]:
class Classifier():

  def build_model(self, shape=(300), optimizer='AdaGrad', activation='relu', units=300, dropout_rate=0.5, learning_rate=0.01, no_layers=1):
    input_1 = Input(shape=shape)
    input_2 = Input(shape=shape)
    inputs = Concatenate(axis=1)([input_1, input_2])
    dense_layer = Dense(units=units, activation=activation)(inputs)

    if no_layers > 1:
      for i in range(no_layers-1):
        dense_layer = Dense(units=units, activation=activation)(dense_layer)
    dropout_layer = Dropout(dropout_rate)(dense_layer)
    dot_layer = Dot(axes=1)([input_1, input_2])
    concat_layer = Concatenate(axis=1)([dropout_layer, dot_layer])
    outputs = Dense(units=1, activation='sigmoid')(concat_layer)

    self.model = Model(inputs=[input_1, input_2], outputs=outputs)
    if optimizer == 'Adam':
      self.model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    elif optimizer == 'AdaGrad':
      self.model.compile(optimizer=Adagrad(), loss='binary_crossentropy', metrics=['accuracy'])
    elif optimizer == 'AdaDelta':
      self.model.compile(optimizer=Adadelta(), loss='binary_crossentropy', metrics=['accuracy'])

  def print_model_summary(self):
    print(self.model.summary())

  def train(self, train, val, epochs, batch_size, hyperparameters, shape):
    optimizer, activation, units, dropout_rate = hyperparameters
    self.build_model(shape, optimizer, activation, units, dropout_rate)

    X_train_1, X_train_2, y_train = train
    X_val_1, X_val_2, y_val = val

    self.history = self.model.fit([X_train_1, X_train_2], y_train, validation_data=([X_val_1, X_val_2], y_val), epochs=epochs, batch_size=batch_size, verbose=1)

  def get_model(self):
    return self.model

  def get_history(self):
    return self.history


In [ ]:
classifier = Classifier()
classifier.build_model()
classifier.print_model_summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 600)          0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 300)          180300      ['concatenate[0][0]']        

In [ ]:
epochs = [50]
batch_sizes = [32]

shape=(300)
max_acc = 0

X_test_1, X_test_2, y_test = test

for epoch in epochs:
  for batch_size in batch_sizes:
    for hyperparameters in hyperparameters_list:
      optimizer, activation, units, dropout_rate = hyperparameters
      print("Epoch {} and Batch Size {}".format(epoch, batch_size))
      print("Optimizer {} Activation {} Units {} Dropout Rate {}".format(optimizer, activation, units, dropout_rate))
      print()
      classifier.train(train, val, epoch, batch_size, hyperparameters, shape)
      model = classifier.get_model()
      loss, accuracy = model.evaluate([X_test_1, X_test_2], y_test)
      if accuracy >= max_acc:
        max_acc = accuracy
        best_model = model
      print()
      print("------------------------------------------------------------")
    print(max_acc)

Epoch 50 and Batch Size 32
Optimizer Adam Activation relu Units 300 Dropout Rate 0.1

Epoch 1/50
526/526 [==============================] - 3s 4ms/step - loss: 0.5116 - accuracy: 0.7500 - val_loss: 0.4629 - val_accuracy: 0.7859
Epoch 2/50
526/526 [==============================] - 3s 5ms/step - loss: 0.4450 - accuracy: 0.7967 - val_loss: 0.4366 - val_accuracy: 0.8062
Epoch 3/50
526/526 [==============================] - 3s 6ms/step - loss: 0.3983 - accuracy: 0.8216 - val_loss: 0.4222 - val_accuracy: 0.8051
Epoch 4/50
526/526 [==============================] - 3s 6ms/step - loss: 0.3473 - accuracy: 0.8519 - val_loss: 0.4133 - val_accuracy: 0.8062
Epoch 5/50
526/526 [==============================] - 2s 4ms/step - loss: 0.3015 - accuracy: 0.8775 - val_loss: 0.4249 - val_accuracy: 0.8158
Epoch 6/50
526/526 [==============================] - 2s 4ms/step - loss: 0.2517 - accuracy: 0.9041 - val_loss: 0.4424 - val_accuracy: 0.8126
Epoch 7/50
526/526 [==============================] - 2s 4ms/s

In [ ]:
def calculate_average_peformance(test):

In [ ]:
del res

In [ ]:
!git clone https://github.com/facebookresearch/LASER.git

Cloning into 'LASER'...
remote: Enumerating objects: 938, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 938 (delta 70), reused 125 (delta 55), pack-reused 787
Receiving objects: 100% (938/938), 2.90 MiB | 11.64 MiB/s, done.
Resolving deltas: 100% (373/373), done.


In [ ]:
!bash LASER/install_external_tools.sh
!bash LASER/install_models.sh

 - creating directory /content/LASER/tools-external
Installing external tools
 - creating directory /content/LASER/tools-external/moses-tokenizer/tokenizer
 - download tokenizer/tokenizer.perl
 - download tokenizer/detokenizer.perl
 - download tokenizer/normalize-punctuation.perl
 - download tokenizer/remove-non-printing-char.perl
 - download tokenizer/deescape-special-chars.perl
 - download tokenizer/lowercase.perl
 - download tokenizer/basic-protected-patterns
 - creating directory /content/LASER/tools-external/moses-tokenizer/share/nonbreaking_prefixes
 - download share/nonbreaking_prefixes/nonbreaking_prefix.ca
 - download share/nonbreaking_prefixes/nonbreaking_prefix.cs
 - download share/nonbreaking_prefixes/nonbreaking_prefix.de
 - download share/nonbreaking_prefixes/nonbreaking_prefix.el
 - download share/nonbreaking_prefixes/nonbreaking_prefix.en
 - download share/nonbreaking_prefixes/nonbreaking_prefix.es
 - download share/nonbreaking_prefixes/nonbreaking_prefix.fi
 - download

In [ ]:
!bash LASER/nllb/download_models.sh

Directory for model download: /content
 - https://dl.fbaipublicfiles.com/nllb/laser/laser2.pt
 - https://dl.fbaipublicfiles.com/nllb/laser/laser2.spm
 - https://dl.fbaipublicfiles.com/nllb/laser/laser2.cvocab
 - https://dl.fbaipublicfiles.com/nllb/laser/laser3-ace_Latn.v1.pt
^C


In [ ]:
!mv laser2.pt LASER/nllb/laser2.pt
!mv laser2.spm LASER/nllb/laser2.spm

In [ ]:
amin = {'haboo': []}

In [ ]:
empty_keys = [key for key, value in amin.items() if len(value) == 0]
for key in empty_keys:
  del amin[key]

In [ ]:
for key, value in amin.items():
  print("{} is {}".format(key, value))